In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import linear_model

from sklearn.feature_selection import RFE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, r2_score, mean_squared_error, auc, mean_absolute_error
from sklearn.model_selection import GridSearchCV, KFold

from sklearn.neural_network import MLPRegressor
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

%matplotlib inline

In [2]:
Salary = pd.read_csv('./Kaggle_Salary.csv')

/home/hanwen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,3,9,11,13,22,24,25,26,27,28,29,45,57,65,84,86,88,108,110,124,126,151,195,209,224,250,263,265,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,305,307,323,326,327,330,342,372,385,386,394,395) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Drop three irrelavent columns
Salary.drop(['Unnamed: 0','index'], axis=1,inplace=True)

QuestionMapping = {}
for col in Salary.columns:
    QuestionMapping[col] = Salary.loc[0,col]
Salary.drop([0],inplace=True)
# print(Salary.shape)
Salary.head()

# See how many null values are in each column
# Here we only deal with those colunmns that has null values more than zero but fewer that 1% of all data
less_001 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (NullNum > 0) & (0.01 * Salary.shape[0] > NullNum):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            less_001.append(col)
# print("We drop nan rows in the below columns:",less_001)

for col in less_001:
    Salary.dropna(subset=[col],inplace=True)
# print(Salary.shape)
between_001_020 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (0.2 * Salary.shape[0] > NullNum) and (NullNum >= 0.01 * Salary.shape[0]):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            between_001_020.append(col)
# print("We use modes in these columns to replace nan value:",between_001_020)
for col in between_001_020:
    Salary[col].fillna(Salary[col].mode()[0],inplace=True)

between_020_050 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if (0.2 * Salary.shape[0]) < NullNum and (NullNum < 0.5 * Salary.shape[0]):
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            between_020_050.append(col)
# print("We set nans in these columns as 'Unknown':",between_020_050)

for col in between_020_050:
    Salary.loc[Salary[col].isnull(),col] = 'Unknown'
more_050 = []
for col in Salary.columns:
    NullNum = Salary[col].isnull().sum()
    if 0.5 * Salary.shape[0] < NullNum:
        if ("Part" not in col) or (col in ['Q39_Part_1','Q39_Part_2']):
            more_050.append(col)
# print("We drop these columns because large part of the data is missing:",more_050)

for col in more_050:
    Salary.drop([col], axis=1,inplace=True)
ALLCOL = Salary.columns.tolist()
def select_col(condition, allcol=ALLCOL):
    digit = len(condition)
    selected = []
    for i in allcol:
        if condition in i:
            if len(i)==digit:
                selected.append(i)
            elif i[digit] not in '0123456789':
                selected.append(i)                
    return selected
DummyCols = ['Q1','Q4','Q5','Q6','Q7','Q10','Q12_MULTIPLE_CHOICE',\
            'Q17','Q18','Q20','Q23','Q26','Q32','Q39_Part_1', 'Q39_Part_2',\
             'Q40','Q48']
for col in DummyCols:
    Salary = pd.get_dummies(data=Salary, columns=[col])
# print(Salary['Q3'].value_counts())
val_counts = Salary['Q3'].value_counts()
value_mask = Salary['Q3'].isin(val_counts.index[val_counts < 50])
Salary.loc[value_mask,'Q3'] = "Other"

Salary = pd.get_dummies(data=Salary, columns=['Q3'])
# print(Salary['Q22'].value_counts())
val_counts = Salary['Q22'].value_counts()
value_mask = Salary['Q22'].isin(val_counts.index[val_counts < 100])
Salary.loc[value_mask,'Q22'] = "Other"

Salary = pd.get_dummies(data=Salary, columns=['Q22'])
def Range_Normalize(value):
    if isinstance(value, str):
        if '-' in value:
            temp = value.split('-')
            return (float(temp[0])+float(temp[1]))/2
        if '+' in value:
            a = value.split('+')
            return float(a[0])
        else:
            return value
    else:
        return value
# Salary[select_col('Q2')]
Salary['Q2_NORMAL'] = Salary['Q2'].apply(Range_Normalize)
Salary.drop(['Q2'],axis=1,inplace=True)
# Salary[select_col('Q8')]
Salary['Q8_NORMAL'] = Salary['Q8'].apply(Range_Normalize)
Salary.drop(['Q8'],axis=1,inplace=True)
# Salary[select_col('Q24')]
def Q24Range_Normalize(value):
    mapdict = {'I have never written code and I do not want to learn':-1,
               'I have never written code but I want to learn':0,'< 1 year':0.5,
              '1-2 years':1.5,'3-5 years':4,'5-10 years':7.5,'10-20 years':15,
               '20-30 years':25,'30-40 years':35,'40+ years':40}
    return mapdict[value]
# Salary[select_col('Q2')]
Salary['Q24_NORMAL'] = Salary['Q24'].apply(Q24Range_Normalize)
Salary.drop(['Q24'],axis=1,inplace=True)
# Salary[select_col('Q25')]
Salary['Q25'].unique()
def Q25Range_Normalize(value):
    mapdict = {'I have never studied machine learning and I do not plan to':-1,
               'I have never studied machine learning but plan to learn in the future':0,
               '< 1 year':0.5,'1-2 years':1.5,'2-3 years':2.5,'3-4 years':3.5,'4-5 years':4.5,
               '5-10 years':7.5,'10-15 years':12.5,'20+ years':20}
    return mapdict[value]

Salary['Q25_NORMAL'] = Salary['Q25'].apply(Q25Range_Normalize)
Salary.drop(['Q25'],axis=1,inplace=True)
def Q41Range_Normalize(value):
    mapdict = {'Very important':10,
               'Slightly important':5,
               'No opinion; I do not know':2,
               'Unknown':0,
               'Not at all important':-10}
    return mapdict[value]

Salary.loc[Salary['Q41_Part_1'].isnull(),'Q41_Part_1'] = 'Unknown'
Salary.loc[Salary['Q41_Part_2'].isnull(),'Q41_Part_2'] = 'Unknown'
Salary.loc[Salary['Q41_Part_3'].isnull(),'Q41_Part_3'] = 'Unknown'

Salary['Q41_Part_1_NORMAL'] = Salary['Q41_Part_1'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_1'],axis=1,inplace=True)
Salary['Q41_Part_2_NORMAL'] = Salary['Q41_Part_2'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_2'],axis=1,inplace=True)
Salary['Q41_Part_3_NORMAL'] = Salary['Q41_Part_3'].apply(Q41Range_Normalize)
Salary.drop(['Q41_Part_3'],axis=1,inplace=True)
def Q43Range_Normalize(value):
    if isinstance(value, str):
        if '-' in value:
            temp = value.split('-')
            return (float(temp[0])+float(temp[1]))/2
        elif value == "0":
            return 0
        else:
            return float("nan")
    else:
        return value
Salary['Q43_NORMAL'] = Salary['Q43'].apply(Q43Range_Normalize)
Salary.drop(['Q43'],axis=1,inplace=True)
Salary['Q43_NORMAL'].fillna(Salary['Q43_NORMAL'].mode()[0],inplace=True)

Salary['Q46_NORMAL'] = Salary['Q46'].apply(Q43Range_Normalize)
Salary.drop(['Q46'],axis=1,inplace=True)
Salary['Q46_NORMAL'].fillna(Salary['Q46_NORMAL'].mode()[0],inplace=True)
# Salary[select_col('Q11')]
import math
def Binarize(value):
    if isinstance(value,str):
        return 1
    else:
        if value == 0 or math.isnan(value):
            return 0
        else:
            return 1
def binarize_col(columns, data=Salary):
    for col in columns:
        data[col] = data[col].apply(Binarize)
NormalCols = select_col('Q11_Part') + select_col('Q13_Part') + \
select_col('Q14_Part') + select_col('Q15_Part') + select_col('Q16_Part')\
+ select_col('Q19_Part') + select_col('Q21_Part') + select_col('Q27_Part')\
+ select_col('Q28_Part') + select_col('Q29_Part') + select_col('Q30_Part') +\
select_col('Q31_Part') + select_col('Q33_Part') + select_col('Q34_Part') +\
select_col('Q36_Part') + select_col('Q38_Part') + select_col('Q42_Part') + \
select_col('Q44_Part') + select_col('Q45_Part') + select_col('Q47_Part') + \
select_col('Q49_Part') + select_col('Q50_Part')
binarize_col(NormalCols)
def TBinarize(value):
    if isinstance(value,str):
        return float(value)
    else:
        if math.isnan(value):
            return -1
        else:
            return value
def Tbinarize_col(columns, data=Salary):
    for col in columns:
        data[col] = data[col].apply(TBinarize)

Tbinarize_col(select_col('Q34_Part'))
Tbinarize_col(select_col('Q35_Part'))
for col in Salary.columns:
    if isinstance(Salary.loc[1,col],str):
        Salary[col] = Salary[col].apply(lambda x:float(x))
Salary.rename(columns={"Q9": "Yearly_compensation"},inplace=True)
Salary.rename(columns={"Time from Start to Finish (seconds)": "TimeUse"},inplace=True)
Salary.drop(Salary[Salary.TimeUse < 180].index,inplace=True)
for col in Salary.columns:
    if len(Salary[col].value_counts())==1:
        Salary.drop([col], axis=1,inplace=True)
Salary.drop(Salary[Salary.Yearly_compensation < 1000].index, inplace=True)
Salary.drop(Salary[Salary.Yearly_compensation > 350000].index, inplace=True)
X = Salary.drop(['Yearly_compensation'],axis=1)
Y = Salary['Yearly_compensation']

from sklearn.model_selection import train_test_split
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y,test_size=0.3, random_state=20)
print(X.shape,X_Train.shape)

(13995, 605) (9796, 605)


In [4]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
def feature_scaling(X_Train, X_Test, method):
    methodmap = {'z-score':StandardScaler(),'min-max':MinMaxScaler(),'robust':RobustScaler()}
    if method not in ['z-score','min-max','robust']:
        print("We don't have this kind of method for normalization. Please choose from ['z-score','min-max','robust']")
        return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
    else:
        ss = methodmap[method].fit(X_Train)
        X_Train = ss.transform(X_Train)
        X_Test = ss.transform(X_Test)
        return pd.DataFrame(X_Train), pd.DataFrame(X_Test)
X_Train_All, X_Test_All = feature_scaling(X_Train,X_Test,'z-score')
print(X_Train_All.shape)

(9796, 605)


In [5]:
# Learner(Vanilla/Basic Model)
from keras.models import Sequential
from keras.layers import Dense
# import sklearn.neural_network as sknn
# import sklearn.model_selection as modsel
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

/home/hanwen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [13]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(13, input_dim=605, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [14]:
estimator = KerasRegressor(build_fn=baseline_model, epochs=1000, batch_size=20, verbose=0)

In [ ]:
# evaluate model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(pipeline, X, Y, cv=kfold)
print("Standardized: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [6]:
model1 = Sequential()

# Stack of Layers
model1.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model1.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model1.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model1.compile(loss='mean_squared_error',optimizer='adam') 

Kmod = model1.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model1.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model1.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1136172527.7065969
-22470222.65762509 243531035.18178013


In [7]:
model2 = Sequential()

# Stack of Layers
model2.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model2.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model2.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model2.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model2.compile(loss='mean_squared_error',optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model2.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model2.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model2.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1156815113.830912
-5073535.138061203 115719352.8687122


In [8]:
model3 = Sequential()

# Stack of Layers
model3.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model3.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model3.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model3.compile(loss='mean_squared_error',optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model3.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model3.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model3.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1110568818.0080972
-4615403.509880863 110371133.38378283


In [9]:
model4 = Sequential()

# Stack of Layers
model4.add(Dense(units=500, kernel_initializer='normal', activation='relu', input_dim=605))
model4.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=200, kernel_initializer='normal', activation='relu'))
model4.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model4.compile(loss='mean_squared_error',optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model4.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model4.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model4.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1108455339.7894738
-5853622.170090846 124297672.87373401


In [10]:
model5 = Sequential()

# Stack of Layers
model5.add(Dense(units=400, kernel_initializer='normal', activation='relu', input_dim=605))
model5.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model5.add(Dense(units=200, kernel_initializer='normal', activation='relu'))
model5.add(Dense(units=100, kernel_initializer='normal', activation='relu'))
model5.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model5.compile(loss='mean_squared_error',optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model5.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model5.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model5.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1136231052.7268398
-4770835.258522127 112214214.26971562


In [11]:
model6 = Sequential()

# Stack of Layers
model6.add(Dense(units=600, kernel_initializer='normal', activation='tanh', input_dim=605))
model6.add(Dense(units=500, kernel_initializer='normal', activation='tanh'))
model6.add(Dense(units=400, kernel_initializer='normal', activation='tanh'))
model6.add(Dense(units=300, kernel_initializer='normal', activation='tanh'))
model6.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model6.compile(loss='mean_squared_error',optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model6.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model6.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model6.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1363606424.8897357
-0.46488337061081286 62180.20387825402


In [15]:
# Training of Model(Train Data is splitted into 80/20)
Kmod = model6.fit(X_Train_All,Y_Train, epochs=3000, batch_size=20, verbose=0)
Error = model6.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model6.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 1468834656.9068825
-1.6002589262059388 82843.59086671998


In [12]:
model7 = Sequential()

# Stack of Layers
model7.add(Dense(units=600, kernel_initializer='normal', activation='relu', input_dim=605))
model7.add(Dense(units=500, kernel_initializer='normal', activation='relu'))
model7.add(Dense(units=400, kernel_initializer='normal', activation='relu'))
model7.add(Dense(units=300, kernel_initializer='normal', activation='relu'))
model7.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model7.compile(loss='mean_squared_error',optimizer='sgd') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model7.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model7.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model7.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 5155631543.266492
-0.9533525525069886 71802.72630012724


In [13]:
model8 = Sequential()

# Stack of Layers
model8.add(Dense(units=600, kernel_initializer='normal', activation='tanh', input_dim=605))
model8.add(Dense(units=500, kernel_initializer='normal', activation='tanh'))
model8.add(Dense(units=400, kernel_initializer='normal', activation='tanh'))
model8.add(Dense(units=300, kernel_initializer='normal', activation='tanh'))
model8.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model8.compile(loss='mean_squared_error',optimizer='sgd') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model8.fit(X_Train_All,Y_Train, epochs=1000, batch_size=5, verbose=0)
Error = model8.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model8.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 5155631543.266492
-0.9533525525069886 71802.72630012724


In [16]:
# Training of Model(Train Data is splitted into 80/20)
Kmod = model8.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model8.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model8.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)


Loss(Based on Testing Set): 5155631543.266492
-0.9533525525069886 71802.72630012724


In [ ]:
model8 = Sequential()

# Stack of Layers
model8.add(Dense(units=600, kernel_initializer='normal', activation='tanh', input_dim=605))
model8.add(Dense(units=500, kernel_initializer='normal', activation='tanh'))
model8.add(Dense(units=400, kernel_initializer='normal', activation='tanh'))
model8.add(Dense(units=300, kernel_initializer='normal', activation='tanh'))
model8.add(Dense(units=1, kernel_initializer='normal', activation='relu'))

model8.compile(loss='mean_squared_error',optimizer='sgd') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model8.fit(X_Train_All,Y_Train, epochs=1000, batch_size=5, verbose=0)
Error = model8.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

Y_Test_pred = model8.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

print(R2,RMSE)

In [32]:
model = Sequential()

# Stack of Layers
model.add(Dense(units=600, activation='tanh', input_dim=605))
model.add(Dense(units=500, activation='tanh'))
model.add(Dense(units=400, activation='tanh'))
model.add(Dense(units=300, activation='tanh'))

model.add(Dense(units=1, activation='relu'))

model.compile(loss='mean_squared_error',
               optimizer='sgd') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)

ValueError: Unknown activation function:identity

In [23]:
model = Sequential()

# Stack of Layers
model.add(Dense(units=600, activation='tanh', input_dim=605))
model.add(Dense(units=500, activation='tanh'))
model.add(Dense(units=400, activation='tanh'))
model.add(Dense(units=300, activation='tanh'))
model.add(Dense(units=200, activation='tanh'))

model.add(Dense(units=1, activation='relu'))

model.compile(loss='mean_squared_error',
               optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model.fit(X_Train_All,Y_Train, epochs=1000, batch_size=20, verbose=0)
Error = model.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)


Loss(Based on Testing Set): 3843737328.758276


In [26]:
model = Sequential()

# Stack of Layers
model.add(Dense(units=600, activation='tanh', input_dim=605))
model.add(Dense(units=500, activation='tanh'))
model.add(Dense(units=400, activation='tanh'))
model.add(Dense(units=300, activation='tanh'))
model.add(Dense(units=200, activation='tanh'))
model.add(Dense(units=200, activation='tanh'))
model.add(Dense(units=200, activation='tanh'))

model.add(Dense(units=1, activation='relu'))

model.compile(loss='mean_squared_error',
               optimizer='adam') 

# Training of Model(Train Data is splitted into 80/20)
Kmod = model.fit(X_Train_All,Y_Train, epochs=2000, batch_size=20, verbose=0)
Error = model.evaluate(X_Test_All, Y_Test, batch_size=50, verbose=0)
print('\nLoss(Based on Testing Set):', Error)


Loss(Based on Testing Set): 3979814794.6387234


In [17]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(hidden_layer_sizes=(600,300,150,75), activation='relu', solver='sgd', alpha=0.0001, \
                   learning_rate_init=0.0002,max_iter=10000, shuffle=True, random_state=20, early_stopping=False, \
                   beta_1=0.9, beta_2=0.999)
# (solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10,8,4), random_state=1, max_iter=1000)
clf.fit(X_Train_All, Y_Train)

/home/hanwen/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(600, 300, 150, 75), learning_rate='constant',
       learning_rate_init=0.0002, max_iter=10000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=20, shuffle=True,
       solver='sgd', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [18]:
Y_Test_pred = clf.predict(X_Test)
R2 = r2_score(Y_Test,Y_Test_pred)
RMSE = np.sqrt(mean_squared_error(Y_Test,Y_Test_pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
RMSE

In [ ]:
R2